# Import Library

In [1]:
import math
import heapq
import pandas as pd
import numpy as np

from random import shuffle
from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Flatten, Multiply, Dropout, Dense, BatchNormalization, Concatenate
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import L1, L2, L1L2

import pickle

#------------------------------------------------------------------------------------------------------------------------------#

from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware

from pydantic import BaseModel

# Data Preparation

## Functions

In [2]:
def get_negatives(uids, iids, items, df_test):
    """
    Returns a pandas dataframe of 20 negative interactions
    based for each user in df_test.
    
    Args:
        uids (np.array): Numpy array of all user ids.
        iids (np.array): Numpy array of all item ids.
        items (list): List of all unique items.
        df_test (dataframe): Our test set.
        
    Returns:
        df_neg (dataframe): dataframe with 100 negative items 
            for each (u, i) pair in df_test.
    """

    negativeList = []
    test_u = df_test['userID'].values.tolist()
    test_i = df_test['vendorID'].values.tolist()

    test_ratings = list(zip(test_u, test_i))
    zipped = set(zip(uids, iids))

    for (u, i) in test_ratings:
        negatives = []
        negatives.append((u, i))
        for t in range(20):
            j = np.random.randint(len(items)) # Get random item id.
            while (u, j) in zipped: # Check if there is an interaction
                j = np.random.randint(len(items)) # If yes, generate a new item id
            negatives.append(j) # Once a negative interaction is found we add it.
        negativeList.append(negatives)

    df_neg = pd.DataFrame(negativeList)

    return df_neg


def mask_first(x):
    """
    Return a list of 0 for the first item and 1 for all others
    """

    result = np.ones_like(x)
    result[0] = 0
    
    return result
   

def train_test_split(df):
    """
    Splits our original data into one test and one
    training set. 
    The test set is made up of one item for each user. This is
    our holdout item used to compute Top@K later.
    The training set is the same as our original data but
    without any of the holdout items.
    
    Args:
        df (dataframe): Our original data
        
    Returns:
        df_train (dataframe): All of our data except holdout items
        df_test (dataframe): Only our holdout items.
    """

    # Create two copies of our dataframe that we can modify
    df_test = df.copy(deep = True)
    df_train = df.copy(deep = True)

    # Group by user_id and select only the first item for
    # each user (our holdout).
    df_test = df_test.groupby(['userID']).first()
    df_test['userID'] = df_test.index
    df_test = df_test[['userID', 'vendorID', 'rating']]

    # Remove the same items for our test set in our training set.
    mask = df.groupby(['userID'])['userID'].transform(mask_first).astype(bool)
    df_train = df.loc[mask]

    return df_train, df_test

In [3]:
def load_dataset():
    """
    Loads the dataset and transforms it into the format we need. 
    We then split it into a training and a test set.
    """

    vendor = pd.read_csv('data/vendor.csv')
    rating = pd.read_csv('data/rating.csv')
    
    vendor = vendor.head(500)

    df = pd.merge(vendor, rating, on = 'vendorID', how = 'inner')
    df = df[['userID', 'vendorID', 'rating']]

    # Create training and test sets.
    df_train, df_test = train_test_split(df)

    # Create lists of all unique users and artists
    users = list(np.sort(df['userID'].unique()))
    items = list(np.sort(df['vendorID'].unique()))

    # Get the rows, columns and values for our matrix.
    rows = df_train['userID'].astype(int)
    cols = df_train['vendorID'].astype(int)

    values = list(df_train['rating'])

    # Get all user ids and item ids.
    uids = np.array(rows.tolist())
    iids = np.array(cols.tolist())

    # Sample negative interactions for each user in our test data
    df_neg = get_negatives(uids, iids, items, df_test)

    return uids, iids, df_train, df_test, df_neg, users, items

## Data

In [4]:
uids, iids, df_train, df_test, df_neg, users, items = load_dataset()

# Model

## Functions

In [5]:
def shuffle_list(U, I, L):
    """
    Shuffle the data U, I, and L in the same order.
    
    Args:
        U (list): All users for every interaction.
        I (list): All items for every interaction.
        L (list): All labels for every interaction.

    Returns:
        shuffled_U (list): Shuffled list of users.
        shuffled_I (list): Shuffled list of items.
        shuffled_L (list): Shuffled list of labels.
    """
    combined = list(zip(U, I, L))
    shuffle(combined)
    shuffled_U, shuffled_I, shuffled_L = zip(*combined)

    return shuffled_U, shuffled_I, shuffled_L


def get_train_instances(uids, iids, items, num_neg = 4):
    """
    Samples a number of negative user-item interactions for each
    user-item pair in our testing data.

    Args:
        uids (list): All users' ID within the historical data
        iids (list): All items' ID within the historical data
        items (list): List of all unique items
     
    Returns:
        user_input (list): A list of all users for each item
        item_input (list): A list of all items for every user,
            both positive and negative interactions.
        labels (list): A list of all labels. 0 or 1.
    """

    user_input, item_input, labels = [], [], []
    zipped = set(zip(uids, iids))

    for (u, i) in zip(uids, iids):
        # Add our positive interaction
        user_input.append(u)
        item_input.append(i)
        labels.append(1)

        # Sample a number of random negative interactions
        for t in range(num_neg):
            j = np.random.randint(len(items))
            while (u, j) in zipped:
                j = np.random.randint(len(items))
            user_input.append(u)
            item_input.append(j)
            labels.append(0)

    return user_input, item_input, labels


def get_hits(k_ranked, holdout):
    """
    Return 1 if the holdout item exists in a given list and 0 if not.
    """

    for item in k_ranked:
        if item == holdout:
            return 1
    return 0


def get_ndcg(ranklist, gtItem):
    """
    NDCG@k is a ranking metric that helps consider 
    both the relevance of items and their positions in the list.
    """

    for i in range(len(ranklist)):
        item = ranklist[i]
        if item == gtItem:
            return math.log(2) / math.log(i+2)
    return 0


def predict_ratings_cf(user_idx, items, model):
    """
    Predict rating score for each user.

    Args:
        user_idx (int): Current index
        items (list): List of all unique items
        model (h5): Tensorflow model
        
    Returns:
        map_item_score (list): predicted current user rating for each item.
    """
    # Prepare user and item arrays for the model.
    predict_user = np.full(len(items), user_idx, dtype = 'int32').reshape(-1, 1)
    np_items = np.array(items).reshape(-1, 1)

    # Predict ratings using the model.
    predictions = model.predict([predict_user, np_items]).flatten().tolist()

    # Map predicted score to item id.
    map_item_score = dict(zip(items, predictions))

    return map_item_score
    

def eval_rating(idx, test_ratings, test_negatives, K, model):
    """
    Generate ratings for the users in our test set and
    check if our holdout item is among the top K highest scores
    and evaluate its position.
    
    Args:
        idx (int): Current index
        test_ratings (list): Our test set user-item pairs
        test_negatives (list): negative items for each
            user in our test set.
        K (int): number of top recommendations
        
    Returns:
        hitrate (list): A list of 1 if the holdout appeared in our
            top K predicted items. 0 if not.
    """
    # Get the negative interactions for our user.
    items = test_negatives[idx]

    # Get the user idx.
    user_idx = test_ratings[idx][0]

    # Get the item idx, i.e., our holdout item.
    holdout = test_ratings[idx][1]

    # Add the holdout to the end of the negative interactions list.
    items.append(holdout)

    # Predict ratings using the model.
    map_item_score = predict_ratings_cf(user_idx, items, model)
    items.pop()

    # Get the K highest ranked items as a list.
    k_ranked = heapq.nlargest(K, map_item_score, key = map_item_score.get)

    # Get a list of hit or no hit.
    hitrate = get_hits(k_ranked, holdout)
    ndcg = get_ndcg(k_ranked, holdout)

    return (hitrate, ndcg)


def evaluate(model, df_test, df_neg, K = 10):
    """
    Calculate the top@K hit ratio for our recommendations.
    
    Args:
        df_neg (dataframe): dataframe containing our holdout items
            and 100 randomly sampled negative interactions for each
            (user, item) holdout pair.
        K (int): The 'K' number of ranked predictions we want
            our holdout item to be present in. 
            
    Returns:
        hits (list): list of "hits". 1 if the holdout was present in 
            the K highest ranked predictions. 0 if not. 
    """

    hitrates = []
    ndcgs = []

    test_u = df_test['userID'].values.tolist()
    test_i = df_test['vendorID'].values.tolist()

    test_ratings = list(zip(test_u, test_i))

    df_neg = df_neg.drop(df_neg.columns[0], axis = 1)
    test_negatives = df_neg.values.tolist()

    for idx in range(len(test_ratings)):
        # For each idx, call eval_one_rating
        (hitrate, ndcg) = eval_rating(idx, test_ratings, test_negatives, K, model)
        hitrates.append(hitrate)
        ndcgs.append(ndcg)

    return (hitrates, ndcgs)

## Structure

In [21]:
# HYPERPARAMS
num_neg = 4
latent_features = 20
epochs = 4
batch_size = 1024
learning_rate = 0.005

In [22]:
# TENSORFLOW GRAPH
# Using the functional API

# Define input layers for user, item, and label.
user_input = Input(shape = (1,), dtype = tf.int32, name = 'user')
item_input = Input(shape = (1,), dtype = tf.int32, name = 'item')
label_input = Input(shape = (1,), dtype = tf.int32, name = 'label')

# User embedding for MLP
mlp_user_embedding = Embedding(input_dim = len(users), 
                               output_dim = latent_features,
                               embeddings_initializer = 'random_normal',
                               embeddings_regularizer = L1(0.01),
                               input_length = 1, 
                               name = 'mlp_user_embedding')(user_input)

# Item embedding for MLP
mlp_item_embedding = Embedding(input_dim = len(items), 
                               output_dim = latent_features,
                               embeddings_initializer = 'random_normal',
                               embeddings_regularizer = L1(0.01),
                               input_length = 1, 
                               name = 'mlp_item_embedding')(item_input)

# User embedding for GMF
gmf_user_embedding = Embedding(input_dim = len(users), 
                               output_dim = latent_features,
                               embeddings_initializer = 'random_normal',
                               embeddings_regularizer = L1(0.01),
                               input_length = 1, 
                               name = 'gmf_user_embedding')(user_input)

# Item embedding for GMF
gmf_item_embedding = Embedding(input_dim = len(items), 
                               output_dim = latent_features,
                               embeddings_initializer = 'random_normal',
                               embeddings_regularizer = L1(0.01),
                               input_length = 1, 
                               name = 'gmf_item_embedding')(item_input)

# GMF layers
gmf_user_flat = Flatten()(gmf_user_embedding)
gmf_item_flat = Flatten()(gmf_item_embedding)
gmf_matrix = Multiply()([gmf_user_flat, gmf_item_flat])

# MLP layers
mlp_user_flat = Flatten()(mlp_user_embedding)
mlp_item_flat = Flatten()(mlp_item_embedding)
mlp_concat = Concatenate()([mlp_user_flat, mlp_item_flat])

mlp_dropout = Dropout(0.1)(mlp_concat)

mlp_layer_1 = Dense(64, 
                    activation = 'relu', 
                    name = 'mlp_layer1')(mlp_dropout)
mlp_batch_norm1 = BatchNormalization(name = 'mlp_batch_norm1')(mlp_layer_1)
mlp_dropout1 = Dropout(0.1, 
                       name = 'mlp_dropout1')(mlp_batch_norm1)

mlp_layer_2 = Dense(32, 
                    activation = 'relu', 
                    name = 'mlp_layer2')(mlp_dropout1)
mlp_batch_norm2 = BatchNormalization(name = 'mlp_batch_norm2')(mlp_layer_2)
mlp_dropout2 = Dropout(0.1, 
                       name = 'mlp_dropout2')(mlp_batch_norm2)

mlp_layer_3 = Dense(16, 
                    activation = 'relu', 
                    kernel_regularizer = L2(0.01),
                    name = 'mlp_layer3')(mlp_dropout2)
mlp_layer_4 = Dense(8, 
                    activation = 'relu', 
                    activity_regularizer = L2(0.01),
                    name = 'mlp_layer4')(mlp_layer_3)

# Merge the two networks
merged_vector = Concatenate()([gmf_matrix, mlp_layer_4])

# Output layer
output_layer = Dense(1, 
                     activation = 'sigmoid',
                     kernel_initializer = 'lecun_uniform',
                     name = 'output_layer')(merged_vector)

# Define the model
modelCF = Model(inputs = [user_input, item_input], outputs = output_layer)

# Compile the model with binary cross entropy loss and Adam optimizer
optimizer = Adam(learning_rate = learning_rate)
modelCF.compile(optimizer = optimizer,
                loss = 'binary_crossentropy',
                metrics = ['accuracy'])

# Print the model summary
modelCF.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 user (InputLayer)           [(None, 1)]                  0         []                            
                                                                                                  
 item (InputLayer)           [(None, 1)]                  0         []                            
                                                                                                  
 mlp_user_embedding (Embedd  (None, 1, 20)                10000     ['user[0][0]']                
 ing)                                                                                             
                                                                                                  
 mlp_item_embedding (Embedd  (None, 1, 20)                10000     ['item[0][0]']          

In [23]:
modelCF = load_model('model/modelCF.h5')
(hitrates, ndcgs) = evaluate(modelCF, df_test, df_neg)
best_hr, best_ndcg = np.array(hitrates).mean(), np.array(ndcgs).mean()

for epoch in range(epochs):
    # Get our training input.
    user_input, item_input, labels = get_train_instances(uids, iids, items, num_neg)

    # Training        
    hist = modelCF.fit([np.array(user_input), np.array(item_input)], #input
                        np.array(labels), # labels 
                        batch_size = batch_size, 
                        verbose = 1, 
                        shuffle = True)

    # Evaluation
    (hitrates, ndcgs) = evaluate(modelCF, df_test, df_neg)
    hr, ndcg, loss = np.array(hitrates).mean(), np.array(ndcgs).mean(), hist.history['loss'][0]
    if hr > best_hr:
        modelCF.save('model/modelCF.h5')
        print("Model has been updated.")

1/1 [==============================] - 0s 28ms/step


In [24]:
# Calculate top@K
modelCF = load_model('model/modelCF.h5') 
hitrates, ndcgs = evaluate(modelCF, df_test, df_neg)
hitrates_avg = ndcgs_avg = np.array(hitrates).mean(), np.array(ndcgs).mean()

with open('model/performance/hitrates_avg.pkl', 'wb') as f:
    pickle.dump(hitrates_avg, f)

with open('model/performance/ndcgs_avg.pkl', 'wb') as f:
    pickle.dump(ndcgs_avg, f)

1/1 [==============================] - 0s 26ms/step


## API

In [25]:
# Load dataset
uids, iids, df_train, df_test, df_neg, users, items = load_dataset()

In [26]:
def model():
    # HYPERPARAMS
    latent_features = 20
    learning_rate = 0.005

    # TENSORFLOW GRAPH
    # Using the functional API

    # Define input layers for user, item, and label.
    user_input = Input(shape = (1,), dtype = tf.int32, name = 'user')
    item_input = Input(shape = (1,), dtype = tf.int32, name = 'item')
    label_input = Input(shape = (1,), dtype = tf.int32, name = 'label')

    # User embedding for MLP
    mlp_user_embedding = Embedding(input_dim = len(users), 
                                output_dim = latent_features,
                                embeddings_initializer = 'random_normal',
                                embeddings_regularizer = L1(0.01),
                                input_length = 1, 
                                name = 'mlp_user_embedding')(user_input)

    # Item embedding for MLP
    mlp_item_embedding = Embedding(input_dim = len(items), 
                                output_dim = latent_features,
                                embeddings_initializer = 'random_normal',
                                embeddings_regularizer = L1(0.01),
                                input_length = 1, 
                                name = 'mlp_item_embedding')(item_input)

    # User embedding for GMF
    gmf_user_embedding = Embedding(input_dim = len(users), 
                                output_dim = latent_features,
                                embeddings_initializer = 'random_normal',
                                embeddings_regularizer = L1(0.01),
                                input_length = 1, 
                                name = 'gmf_user_embedding')(user_input)

    # Item embedding for GMF
    gmf_item_embedding = Embedding(input_dim = len(items), 
                                output_dim = latent_features,
                                embeddings_initializer = 'random_normal',
                                embeddings_regularizer = L1(0.01),
                                input_length = 1, 
                                name = 'gmf_item_embedding')(item_input)

    # GMF layers
    gmf_user_flat = Flatten()(gmf_user_embedding)
    gmf_item_flat = Flatten()(gmf_item_embedding)
    gmf_matrix = Multiply()([gmf_user_flat, gmf_item_flat])

    # MLP layers
    mlp_user_flat = Flatten()(mlp_user_embedding)
    mlp_item_flat = Flatten()(mlp_item_embedding)
    mlp_concat = Concatenate()([mlp_user_flat, mlp_item_flat])

    mlp_dropout = Dropout(0.1)(mlp_concat)

    mlp_layer_1 = Dense(64, 
                        activation = 'relu', 
                        name = 'mlp_layer1')(mlp_dropout)
    mlp_batch_norm1 = BatchNormalization(name = 'mlp_batch_norm1')(mlp_layer_1)
    mlp_dropout1 = Dropout(0.1, 
                        name = 'mlp_dropout1')(mlp_batch_norm1)

    mlp_layer_2 = Dense(32, 
                        activation = 'relu', 
                        name = 'mlp_layer2')(mlp_dropout1)
    mlp_batch_norm2 = BatchNormalization(name = 'mlp_batch_norm2')(mlp_layer_2)
    mlp_dropout2 = Dropout(0.1, 
                        name = 'mlp_dropout2')(mlp_batch_norm2)

    mlp_layer_3 = Dense(16, 
                        activation = 'relu', 
                        kernel_regularizer = L2(0.01),
                        name = 'mlp_layer3')(mlp_dropout2)
    mlp_layer_4 = Dense(8, 
                        activation = 'relu', 
                        activity_regularizer = L2(0.01),
                        name = 'mlp_layer4')(mlp_layer_3)

    # Merge the two networks
    merged_vector = Concatenate()([gmf_matrix, mlp_layer_4])

    # Output layer
    output_layer = Dense(1, 
                        activation = 'sigmoid',
                        kernel_initializer = 'lecun_uniform',
                        name = 'output_layer')(merged_vector)

    # Define the model
    modelCF = Model(inputs = [user_input, item_input], outputs = output_layer)

    # Compile the model with binary cross entropy loss and Adam optimizer
    optimizer = Adam(learning_rate = learning_rate)
    modelCF.compile(optimizer = optimizer,
                    loss = 'binary_crossentropy',
                    metrics = ['accuracy'])
    
    return modelCF


def train_model_CF():
    # HYPERPARAMS
    num_neg = 4
    epochs = 4
    batch_size = 1024

    prev_modelCF = load_model('model/modelCF.h5')
    try:
        with open('model/performance/hitrates_avg_CF.pkl', 'rb') as f:
            best_hr = pickle.load(f)
        with open('model/performance/ndcgs_avg_CF.pkl', 'rb') as f:
            best_ndcgs = pickle.load(f)
    except:
        best_hr = 0
        best_ndcgs = 0

    curr_modelCF = model()
    for epoch in range(epochs):
        # Get our training input.
        user_input, item_input, labels = get_train_instances(uids, iids, items, num_neg)
    
        # Training        
        hist = curr_modelCF.fit([np.array(user_input), np.array(item_input)], #input
                                np.array(labels), # labels 
                                batch_size = batch_size, 
                                verbose = 0, 
                                shuffle = True)

        # Evaluation
        (hitrates, ndcgs) = evaluate(curr_modelCF, df_test, df_neg)
        hr, ndcg, loss = np.array(hitrates).mean(), np.array(ndcgs).mean(), hist.history['loss'][0]
        if hr > best_hr:
            curr_modelCF.save('model/modelCF.h5')
            
            with open('model/performance/hitrates_avg.pkl', 'wb') as f:
                pickle.dump(hitrates_avg, f)

            with open('model/performance/ndcgs_avg.pkl', 'wb') as f:
                pickle.dump(ndcgs_avg, f)

            print("Model has been updated.")

In [27]:
train_model_CF()

1/1 [==============================] - 0s 26ms/step
Model has been updated.


c:\Users\c640\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 0s 27ms/step
Model has been updated.


c:\Users\c640\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 0s 65ms/step
Model has been updated.


c:\Users\c640\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 0s 37ms/step
Model has been updated.


c:\Users\c640\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [30]:
def get_top_k_items(dct, k = 10):
    # Use nlargest to get the top n key-value pairs based on values.
    top_k_items = heapq.nlargest(k, dct.items(), key = lambda item: item[1])

    return top_k_items

In [29]:
origins = [
    "http://localhost",
    "http://localhost:3000",
]

app.add_middleware(
    CORSMiddleware,
    allow_origins = origins,
    allow_credentials = True,
    allow_methods = ["*"],
    allow_headers = ["*"],
)


@app.get("/")
async def root():
    return {"message": "Hello World"}


class Recommendation(BaseModel):
    userID: str


@app.post('/recsys/train')
def train():
    train_model_CF()
    '''
    train_model_CB()
    '''

    return 'Model trained successfully.'

@app.post('/recsys/recommend')
async def recommendItem(req: Recommendation):
    userID = req.userID

    # Load data, CF model, and its perforamnce
    uids, iids, df_train, df_test, df_neg, users, items = load_dataset()
    modelCF = load_model('model/modelCF.h5')
    with open('model/performance/hitrates_avg_CF.pkl', 'wb') as f:
        hitrates_avg = pickle.load(f)
    with open('model/performance/ndcgs_avg_CF.pkl', 'wb') as f:
        ndcgs_avg = pickle.load(f)
    
    try:
        # Predict ratings using CF model
        cf_works = True
        ratingsCF = predict_ratings_cf(
            user_idx = userID,
            items = items,
            model = modelCF
        )
        
        recommendations = get_top_k_items(ratingsCF, k = 10)
        average_ratings = np.mean([value for key, value in recommendations.items()])
    
        # If predicted rating avg. is less than 4 or HR@10 is less than 0.4 then CF model fails
        if (average_ratings >= 4) or (hitrates_avg >= 0.4) or (ndcgs_avg >= 0.3):
            recommendation_items = [key for key, value in recommendations.items()]
    
        # Else proceed to predict using CB model
        cf_works = False
        
        '''
        FILL THE CODE HERE FOR CB
        '''

        dct = {
            'status': 200,
            'message': "recommendation for user has been successfully get.",
            'data': {
                'user_id': userID,
                'recommendations': recommendation_items
                },
            'success': True,
            'error': None
        }

    except Exception as e:
        recommendation_items = []
        dct = {
            'status': 404,
            'message': "recommendation for user has failed.",
            'data': {
                'user_id': userID,
                'recommendations': recommendation_items
                },
            'success': False,
            'error': e
        }

    return dct

NameError: name 'app' is not defined